<a href="https://colab.research.google.com/github/haroonwaheed19/Decision-Tree-Regressor-from-Scratch/blob/main/Decision_Tree_Regressor_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
class Node:
  def __init__(self,feature_index=None,threshold=None,left=None,right=None,info_gain=None,value=None):
    #for decision node
    self.feature_index = feature_index
    self.threshold = threshold
    self.left = left
    self.right = right
    self.info_gain = info_gain

    #for leaf node
    self.value = value

In [ ]:
class DecisionTreeRegressor:
  # Constructor
    def __init__(self, min_samples_split=2, max_depth=2):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth

  # Fit function for model Training
    def fit(self, X, Y):
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

  # Helper Function to build the tree
    def build_tree(self, dataset, curr_depth=0):
        X, Y = dataset[:, :-1], dataset[:, -1]
        num_samples, num_features = np.shape(X)

        # Split untill the stopping conditions met
        if num_samples >= self.min_samples_split and curr_depth <= self.max_depth:
          # finding best split for the dataset division
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # checking if info gain is not null
            if best_split["info_gain"] > 0:
              # creating left subtree recursively
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth + 1)
              # creating right subtree recursively
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth + 1)
              # return the decision node
                return Node(best_split["feature_index"], best_split["threshold"],
                            left_subtree, right_subtree, best_split["info_gain"])

      # leaf node calculation if stopping condition fails
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)

  # calculate leaf value by calculating the mean for labels
    def calculate_leaf_value(self, Y):
        return np.mean(Y)

  # best split method to split the data and calculate info-gain
    def get_best_split(self, dataset, num_samples, num_features):
      # empty directory to store the best split
        best_split = {}
        # initializing the max info gain with negative infinity
        max_info_gain = -float("inf")

      # looping on all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)

            # loop over all the thresholds of that feature
            for threshold in possible_thresholds:
              # splitting the data
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)

                # checking if the child is not empty than procceed
                if len(dataset_left) > 0 and len(dataset_right) > 0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]

                    # calculating info gain
                    current_info_gain = self.information_gain(y, left_y, right_y)

                    # storing the values in the best split if current info is higher than max-info
                    if current_info_gain > max_info_gain:
                        best_split = {
                            "feature_index": feature_index,
                            "threshold": threshold,
                            "dataset_left": dataset_left,
                            "dataset_right": dataset_right,
                            "info_gain": current_info_gain
                        }
                        # updating the max info gain
                        max_info_gain = current_info_gain
      # return best split
        return best_split


    # splitting the data set into left and right node
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index] <= threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index] > threshold])
        return dataset_left, dataset_right

    # calculating info gain using variance reduction
    def information_gain(self, parent, l_child, r_child):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)

        gain = self.variance(parent) - (weight_l * self.variance(l_child) + weight_r * self.variance(r_child))
        return gain

    # variance calculation
    def variance(self, y):
        return np.var(y)

    # predict function
    def predict(self, X):
        return [self.make_prediction(x, self.root) for x in X]

    # helper function to recursively find the prediction from root tree
    def make_prediction(self, x, tree):
        if tree.value is not None:
            return tree.value

        feature_val = x[tree.feature_index]
        if feature_val <= tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

  # print tree function
    def print_tree(self, tree=None, indent=" "):
        if tree is None:
            tree = self.root

        if tree.value is not None:
            print(round(tree.value, 3))  # Round regression values
        else:
            print("X_" + str(tree.feature_index), "<=", tree.threshold, ":", round(tree.info_gain, 3))
            print(f"{indent}left:", end="")
            self.print_tree(tree.left, indent + indent)
            print(f"{indent}right:", end="")
            self.print_tree(tree.right, indent + indent)